# Lecture 22: Cost Functions

## Load Packages

In [ ]:
%matplotlib inline
import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms
from skimage.measure import compare_ssim as ssim #Structural similarity index

print(torch.__version__) # This code has been updated for PyTorch 1.0.0

## Load Data

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
BatchSize = 100

trainset = torchvision.datasets.MNIST(root='./MNIST', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BatchSize,
                                          shuffle=True, num_workers=4) # Creating dataloader

testset = torchvision.datasets.MNIST(root='./MNIST', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BatchSize,
                                         shuffle=False, num_workers=4) # Creating dataloader

In [ ]:
# Check availability of GPU

use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')
    device = "cuda"
else:
    print('GPU is not available!')
    device = "cpu"

# Regression Losses
## Define the Autoencoder

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.Tanh())
        self.decoder = nn.Sequential(          
            nn.Linear(400, 28*28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

net1 = autoencoder().to(device) # Network to be trained using MSE loss
net2 = autoencoder().to(device) # Network to be trained using L1 loss

## Train Autoencoder

In [ ]:
def Train(model,optimizer,criterion,datainput,label):
    net.train()
    optimizer.zero_grad()
    output = model(datainput)
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()
    return loss.item()

In [ ]:
iterations = 10
learning_rate = 1

optimizer1 = optim.Adam(net1.parameters(), lr=1e-4) # Network to be trained using MSE loss
optimizer2 = optim.Adam(net2.parameters(), lr=1e-4) # Network to be trained using L1 loss

criterion1 = nn.MSELoss()
criterion2 = nn.L1Loss()

Plotssim1 = []
Plotssim2 = []
plotLoss1 = []
plotLoss2 = []

testImage = testloader.dataset[0][0]
testinputs = testImage.view(-1, 28*28).to(device)

for epoch in range(iterations):  # loop over the dataset multiple times
    running_loss1 = 0.0
    running_loss2 = 0.0

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs = inputs.view(-1, 28*28).to(device)              
      
        trainLoss1  = Train(net1,optimizer1,criterion1,inputs,inputs) # MSE Loss
        trainLoss2 = Train(net2,optimizer2,criterion2,inputs,inputs)  # L1 Loss  
        
        running_loss1 += trainLoss1
        running_loss2 += trainLoss2
    plotLoss1.append(running_loss1/(i+1))
    plotLoss2.append(running_loss2/(i+1))       

    net1.eval()  
    net2.eval()
    with torch.no_grad():
        outputs = net1(testinputs.to(device))    
        if use_gpu:
            outputs = outputs.cpu()
            testinputs = testinputs.cpu()
        ssim1 = ssim(outputs.data.view(28,28).numpy(),testinputs.data.view(28,28).numpy())
    
        outputs = net2(testinputs.to(device))
        if use_gpu:
            outputs = outputs.cpu()
            testinputs = testinputs.cpu()
        ssim2 = ssim(outputs.data.view(28,28).numpy(),testinputs.data.view(28,28).numpy())

    Plotssim1.append(float(ssim1))
    Plotssim2.append(float(ssim2))
    
    print('At Epoch '+str(epoch+1))
    print('With MSELoss: Loss = {:.6f}, SSIM Index = {:.5f} '.format(running_loss1/(i+1),float(ssim1)))
    print('With L1Loss: Loss = {:.6f}, SSIM Index = {:.5f} '.format(running_loss2/(i+1),float(ssim2)))

fig = plt.figure()        
plt.plot(range(epoch+1),plotLoss1,'r-',label='Mean Square Error')
plt.plot(range(epoch+1),plotLoss2,'g-',label='L1 Loss')   
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Training Loss')  

fig = plt.figure()         
plt.plot(range(epoch+1),Plotssim1,'r-',label='SSIM Index (MSE)')
plt.plot(range(epoch+1),Plotssim2,'g-',label='SSIM Index (L1)')      
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Testing SSIM') 

# Classification Loss
## Neural Network

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.Layer1 = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.ReLU(),
            nn.Linear(400, 256),
            nn.ReLU())
        self.Layer2 = nn.Sequential(
            nn.Linear(256, 10))

    def forward(self, x):
        x = self.Layer1(x)
        x = self.Layer2(x)
        return x

In [ ]:
net1 = NeuralNet().to(device) # Network to be trained using cross-entropy loss
net2 = NeuralNet().to(device) # Network to be trained using NLL loss
net3 = NeuralNet().to(device) # Network to be trained using multi-margin loss

## Train Classifier

In [ ]:
def Train(model,optimizer,criterion,datainput,label,lossType):
    model.train()
    optimizer.zero_grad()
    output = model(datainput)
    if lossType == 'NLL':
        loss = criterion(F.log_softmax(output,dim=1), label)
    else:
        loss = criterion(output, label)
    loss.backward()
    optimizer.step()
    return loss.item()

In [ ]:
iterations = 10
learning_rate = 0.1
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.NLLLoss()
criterion3 = nn.MultiMarginLoss()

optimizer1 = optim.Adam(net1.parameters(), lr=1e-4) # Network to be trained using cross-entropy loss
optimizer2 = optim.Adam(net2.parameters(), lr=1e-4) # Network to be trained using NLL loss
optimizer3 = optim.Adam(net3.parameters(), lr=1e-4) # Network to be trained using multi-margin loss

Plotacc1 = []
Plotacc2 = []
Plotacc3 = []

plotLoss1 = []
plotLoss2 = []
plotLoss3 = []

for epoch in range(iterations):  # loop over the dataset multiple times

    correct1 = 0
    correct2 = 0
    correct3 = 0
    runningLoss1 = 0
    runningLoss2 = 0
    runningLoss3 = 0
    total = 0
    for i, data in enumerate(trainloader, 0): # i-> batch count
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.view(-1, 28*28).to(device), labels.to(device)    
        
        trainLoss1 = Train(net1,optimizer1,criterion1,inputs,labels,lossType='CE')
        trainLoss2 = Train(net2,optimizer2,criterion2,inputs,labels,lossType='NLL')   
        trainLoss3 = Train(net3,optimizer3,criterion3,inputs,labels,lossType='MM')    

        runningLoss1 += trainLoss1
        runningLoss2 += trainLoss2
        runningLoss3 += trainLoss3
   
    runningLoss1 = runningLoss1/(i+1)
    runningLoss2 = runningLoss2/(i+1)
    runningLoss3 = runningLoss3/(i+1)          
   
    plotLoss1.append(runningLoss1)
    plotLoss2.append(runningLoss2)
    plotLoss3.append(runningLoss3)
    
    net1.eval()
    net2.eval()
    net3.eval()
    with torch.no_grad():    
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.view(-1, 28*28).to(device), labels.to(device)
            total += labels.size(0)

            outputs = net1(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct1 += (predicted == labels).sum()

            outputs = net2(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct2 += (predicted == labels).sum()

            outputs = net3(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct3 += (predicted == labels).sum()

    Plotacc1.append(100*float(correct1)/float(total))
    Plotacc2.append(100*float(correct2)/float(total))
    Plotacc3.append(100*float(correct3)/float(total))
    
    print('At Epoch '+str(epoch+1))
    print('With CrossEntropyLoss: Loss = {:.6f} , Acc = {:.4f}%'.format(runningLoss1,100*float(correct1)/float(total)))
    print('With NegativeLogLikelihoodLoss: Loss = {:.6f} , Acc = {:.4f}%'.format(runningLoss2,100*float(correct2)/float(total)))
    print('With MultiMarginLoss: Loss = {:.6f} , Acc = {:.4f}%\n'.format(runningLoss3,100*float(correct3)/float(total)))
    
fig = plt.figure()        
plt.plot(range(epoch+1),plotLoss1,'r-',label='Cross Entropy Loss')
plt.plot(range(epoch+1),plotLoss2,'g-',label='Negative Log Likelihood Loss')   
plt.plot(range(epoch+1),plotLoss3,'b-',label='Multi Margin Loss')  
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Training Loss')  
    
fig = plt.figure()        
plt.plot(range(epoch+1),Plotacc1,'r-',label='Cross Entropy Loss')
plt.plot(range(epoch+1),Plotacc2,'g-',label='Negative Log Likelihood Loss')   
plt.plot(range(epoch+1),Plotacc3,'b-',label='Multi Margin Loss')  
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Testing Accuracy')  
print('Finished Training')
